In [5]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

In [75]:
def report(classifier, X_test, y_test, target_names):
    y_predicted = classifier.predict(X_test)

    print(metrics.classification_report(y_test, y_predicted,
                                        target_names=target_names))
    cm = metrics.confusion_matrix(y_test, y_predicted)
    print(cm)

In [67]:
def grid_search(pipeline, parameters, X_train, y_train):
    gsc = GridSearchCV(
        pipeline,
        parameters,
        n_jobs=-1
    )
    gsc = gsc.fit(X_train, y_train)
    print("Best score: ", gsc.best_score_)
    print("Best parameters: ")
    for param_name in sorted(parameters.keys()):
        print("%s: %r" % (param_name, gsc.best_params_[param_name]))
    return gsc

In [68]:
movie_reviews_data_folder = r"./data"
dataset = load_files(movie_reviews_data_folder, shuffle=False)
print('Size:', len(dataset.data))
print('Target names:', dataset.target_names)

Size: 2000
Target names: ['neg', 'pos']


Split test and train data

In [87]:
X_train, X_test, y_train, y_test = train_test_split(
    dataset.data, 
    dataset.target, 
    test_size=0.25, 
    random_state=None
)

### MultinomialNB

1. Escreva *pipeline de classificação de texto* para classificar reviews de filmes como positivos e negativos;

In [70]:
pipeline = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
])

2. Encontre um bom conjunto de parâmetros utilizando `GridSearchCV`;

In [85]:
parameters = {'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
              'tfidf__use_idf': (True, False),
              'clf__alpha': (0, 0.5, 1.0),
              'clf__fit_prior': (True, False),
}

3. Avalie o classificador utilizando parte do conjunto de dados (previamente separado para testes).

In [88]:
classifier = grid_search(pipeline, parameters, X_train, y_train)
results = report(classifier, X_test, y_test, dataset.target_names)
results

Best score:  0.8306666666666664
Best parameters: 
clf__alpha: 0.5
clf__fit_prior: True
tfidf__use_idf: True
vect__ngram_range: (1, 2)
              precision    recall  f1-score   support

         neg       0.85      0.81      0.83       251
         pos       0.82      0.86      0.84       249

    accuracy                           0.83       500
   macro avg       0.83      0.83      0.83       500
weighted avg       0.83      0.83      0.83       500

[[204  47]
 [ 36 213]]


### LinearSVC

1. Escreva pipeline de classificação de texto para classificar reviews de filmes como positivos e negativos;

In [89]:
pipeline = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LinearSVC()),
])

2. Encontre um bom conjunto de parâmetros utilizando `GridSearchCV`;

In [96]:
parameters = {'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
              'tfidf__use_idf': (True, False),
              'clf__loss': ("hinge", "squared_hinge"),
              'clf__dual': (True, False),
              'clf__fit_intercept': (True, False),
}

3. Avalie o classificador utilizando parte do conjunto de dados (previamente separado para testes).

In [97]:
classifier = grid_search(pipeline, parameters, X_train, y_train)
results = report(classifier, X_test, y_test, dataset.target_names)
results

/Users/patrick/.local/pipx/venvs/jupyter/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Best score:  0.8460000000000001
Best parameters: 
clf__dual: True
clf__fit_intercept: False
clf__loss: 'hinge'
tfidf__use_idf: True
vect__ngram_range: (1, 1)
              precision    recall  f1-score   support

         neg       0.85      0.83      0.84       251
         pos       0.84      0.86      0.85       249

    accuracy                           0.84       500
   macro avg       0.84      0.84      0.84       500
weighted avg       0.84      0.84      0.84       500

[[209  42]
 [ 36 213]]


## Conclusão

Optei por utilizar os algoritmos de classificação MultinomialNB e o LinearSVC, apesar de ter utilizado em ambos as features CountVectorizer e TfidTransformer, com parametros diferentes nas pipelines, o resultado foi bem próximo um do outro. Enquanto o MultinomialNB teve seu melhor resultado com 83% o LinearSVC teve um resultado de 84% acertos. 

Utilizei a função GridSearchCV para encontrar o melhor conjunto de parametros para cada uma das features. 

Para o MultinomialNB a melhor combinação foi a seguinte:
- clf__alpha: 0.5
- clf__fit_prior: True
- tfidf__use_idf: True
- vect__ngram_range: (1, 2)

Para o LinerSVC a melhor combinação foi a seguinte:
- clf__dual: True
- clf__fit_intercept: False
- clf__loss: 'hinge'
- tfidf__use_idf: True
- vect__ngram_range: (1, 1)

No relatório de classificação é possível ver que eles tiveram o mesmo resultado para a precisão de negativos e recall de positivos. Porém na precisão de positivos e recall de negativos o algoritmos LinearSVC se sai melhor.

Na matriz de confusão podemos ver que ambos algoritmos classificaram os verdadeiros positivos 213 das vezes, enquanto o LinearSVC se sai melhor na classificação de verdadeiros negativos.